<a href="https://colab.research.google.com/github/WoojinJeonkr/DeepLearning/blob/main/Pretraining_BERT_with_Hugging_Face_Transformers.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Hugging Face Transformers를 사용한 BERT 사전 훈련
- 내용 출처: [Pretraining BERT with Hugging Face Transformers](https://keras.io/examples/nlp/pretraining_BERT/)
- 목표: Datasets에서 로드된 영어 데이터 세트의 Transformers를 사용하여 MLM 및 NSP 객체를 최적화하는 BERT 사전 훈련


## 01. BERT
- 변압기의 양방향 인코더 표현
- 텍스트에서 단어(또는 하위 단어) 간의 컨텍스트 관계를 학습하는 주의 메커니즘인 Transformer(전체 단어 시퀀스를 한 번에 읽음)를 사용
- BERT의 목표: 언어 모델 생성
- 2가지 훈련 전략 사용
1. 마스크 언어 모델링(MLM): BERT에 제공하기 전에 각 시퀀스의 단어 중 15%가 [MASK]토큰으로 대체된 뒤 모델은 시퀀스의 마스크되지 않은 다른 단어가 제공하는 컨텍스트를 기반으로 마스크된 단어의 원래 값을 예측하려고 시도
2. 다음 문장 예측(NSP): 모델은 문장 쌍을 입력으로 받고 쌍의 두 번째 문장이 원본 문서의 후속 문장인지 예측하는 방법을 학습   
&nbsp;훈련 중에 입력의 50%는 두 번째 문장이 원본 문서의 후속 문장인 쌍이고   
&nbsp;나머지 50%에서는 말뭉치에서 임의의 문장이 두 번째 문장으로 선택되며 임의의 문장이 첫 번째 문장과의 단절을 나타낼 것이라고 가정


## 02. 요구사항 설치

In [1]:
!pip install git+https://github.com/huggingface/transformers.git
!pip install datasets
!pip install huggingface-hub
!pip install nltk

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/huggingface/transformers.git to /tmp/pip-req-build-3l8e8z5_
  Running command git clone -q https://github.com/huggingface/transformers.git /tmp/pip-req-build-3l8e8z5_
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 6.6 MB 3.0 MB/s 
     |████████████████████████████████| 120 kB 38.3 MB/s 
  Created wheel for transformers: filename=transformers-4.22.0.dev0-py3-none-any.whl size=4742652 sha256=c8de774a3e873bc6a7f7bbe2eac1f2f538009e3971afecb199c0bc5b8531eb00
  Stored in directory: /tmp/pip-ephem-wheel-cache-2fvj9uqy/wheels/90/a5/44/6bcd83827c8a60628c5ad602f429cd5076bcce5f2a90054947
Successfully built transformers
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |███████████████████████████████

## 03.필요한 라이브러리 불러오기

In [2]:
import nltk
import random
import logging

import tensorflow as tf
from tensorflow import keras

nltk.download("punkt")
tf.get_logger().setLevel(logging.ERROR)
tf.keras.utils.set_random_seed(42)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


## 04. 매개변수 정의

In [3]:
TOKENIZER_BATCH_SIZE = 256
TOKENIZER_VOCABULARY = 25000
BLOCK_SIZE = 128
NSP_PROB = 0.50
SHORT_SEQ_PROB = 0.1
MAX_LENGTH = 512

MLM_PROB = 0.2

TRAIN_BATCH_SIZE = 2
MAX_EPOCHS = 1
LEARNING_RATE = 1e-4

MODEL_CHECKPOINT = "bert-base-cased"

## 05. WikiText 데이터세트 로드

In [4]:
from datasets import load_dataset
dataset = load_dataset("wikitext", "wikitext-2-raw-v1")

Generating test split:   0%|          | 0/4358 [00:00<?, ? examples/s]

Generating train split:   0%|          | 0/36718 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/3760 [00:00<?, ? examples/s]

Dataset wikitext downloaded and prepared to /root/.cache/huggingface/datasets/wikitext/wikitext-2-raw-v1/1.0.0/a241db52902eaf2c6aa732210bead40c090019a499ceb13bcbfa3f8ab646a126. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

In [5]:
# 데이터세트 확인
print(dataset)

DatasetDict({
    test: Dataset({
        features: ['text'],
        num_rows: 4358
    })
    train: Dataset({
        features: ['text'],
        num_rows: 36718
    })
    validation: Dataset({
        features: ['text'],
        num_rows: 3760
    })
})


## 06. 토크나이저 훈련

In [6]:
# wiki text 말뭉치의 모든 원시 문서 목록 생성
all_texts = [
    doc for doc in dataset["train"]["text"] if len(doc) > 0 and not doc.startswith(" =")
]

In [7]:
# batch iterator 토크나이저 훈련에 도움이 되는 함수 생성
def batch_iterator():
    for i in range(0, len(all_texts), TOKENIZER_BATCH_SIZE):
        yield all_texts[i : i + TOKENIZER_BATCH_SIZE]

In [8]:
# 모델로 사용하려는 토크나이저 불러오기
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained(MODEL_CHECKPOINT)

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


Moving 0 files to the new cache system


0it [00:00, ?it/s]

Downloading:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/213k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/436k [00:00<?, ?B/s]

In [9]:
# 토크나이저 훈련
tokenizer = tokenizer.train_new_from_iterator(
    batch_iterator(), vocab_size=TOKENIZER_VOCABULARY
)

## 07. 데이터 전처리

In [10]:
# 훈련/검증 데이터 분할
dataset["train"] = dataset["train"].select([i for i in range(1000)])
dataset["validation"] = dataset["validation"].select([i for i in range(1000)])

In [11]:
max_num_tokens = BLOCK_SIZE - tokenizer.num_special_tokens_to_add(pair=True)

def prepare_train_features(examples):
    examples["document"] = [
        d.strip() for d in examples["text"] if len(d) > 0 and not d.startswith(" =")
    ]
    examples["sentences"] = [
        nltk.tokenize.sent_tokenize(document) for document in examples["document"]
    ]
    examples["tokenized_sentences"] = [
        [tokenizer.convert_tokens_to_ids(tokenizer.tokenize(sent)) for sent in doc]
        for doc in examples["sentences"]
    ]

    examples["input_ids"] = []
    examples["token_type_ids"] = []
    examples["attention_mask"] = []
    examples["next_sentence_label"] = []

    for doc_index, document in enumerate(examples["tokenized_sentences"]):

        current_chunk = []
        current_length = 0
        i = 0
        target_seq_length = max_num_tokens

        if random.random() < SHORT_SEQ_PROB:
            target_seq_length = random.randint(2, max_num_tokens)

        while i < len(document):
            segment = document[i]
            current_chunk.append(segment)
            current_length += len(segment)
            if i == len(document) - 1 or current_length >= target_seq_length:
                if current_chunk:
                    a_end = 1
                    if len(current_chunk) >= 2:
                        a_end = random.randint(1, len(current_chunk) - 1)

                    tokens_a = []
                    for j in range(a_end):
                        tokens_a.extend(current_chunk[j])

                    tokens_b = []

                    if len(current_chunk) == 1 or random.random() < NSP_PROB:
                        is_random_next = True
                        target_b_length = target_seq_length - len(tokens_a)

                        for _ in range(10):
                            random_document_index = random.randint(
                                0, len(examples["tokenized_sentences"]) - 1
                            )
                            if random_document_index != doc_index:
                                break

                        random_document = examples["tokenized_sentences"][
                            random_document_index
                        ]
                        random_start = random.randint(0, len(random_document) - 1)
                        for j in range(random_start, len(random_document)):
                            tokens_b.extend(random_document[j])
                            if len(tokens_b) >= target_b_length:
                                break
                        num_unused_segments = len(current_chunk) - a_end
                        i -= num_unused_segments
                    else:
                        is_random_next = False
                        for j in range(a_end, len(current_chunk)):
                            tokens_b.extend(current_chunk[j])

                    input_ids = tokenizer.build_inputs_with_special_tokens(
                        tokens_a, tokens_b
                    )
                    token_type_ids = tokenizer.create_token_type_ids_from_sequences(
                        tokens_a, tokens_b
                    )

                    padded = tokenizer.pad(
                        {"input_ids": input_ids, "token_type_ids": token_type_ids},
                        padding="max_length",
                        max_length=MAX_LENGTH,
                    )

                    examples["input_ids"].append(padded["input_ids"])
                    examples["token_type_ids"].append(padded["token_type_ids"])
                    examples["attention_mask"].append(padded["attention_mask"])
                    examples["next_sentence_label"].append(1 if is_random_next else 0)
                    current_chunk = []
                    current_length = 0
            i += 1

    del examples["document"]
    del examples["sentences"]
    del examples["text"]
    del examples["tokenized_sentences"]

    return examples

tokenized_dataset = dataset.map(
    prepare_train_features, batched=True, remove_columns=["text"], num_proc=1,
)

  0%|          | 0/5 [00:00<?, ?ba/s]

You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

In [12]:
# MLM을 위한 데이터를 준비하기 위해 NSP 작업에 대해 이미 준비된 데이터 세트의 Transformers 라이브러리에서 제공 하는 collator호출 사용
from transformers import DataCollatorForLanguageModeling

collater = DataCollatorForLanguageModeling(
    tokenizer=tokenizer, mlm=True, mlm_probability=MLM_PROB, return_tensors="tf"
)

In [13]:
# 모델을 훈련하는 훈련 세트 정의
train = tokenized_dataset["train"].to_tf_dataset(
    columns=["input_ids", "token_type_ids", "attention_mask"],
    label_cols=["labels", "next_sentence_label"],
    batch_size=TRAIN_BATCH_SIZE,
    shuffle=True,
    collate_fn=collater,
)

validation = tokenized_dataset["validation"].to_tf_dataset(
    columns=["input_ids", "token_type_ids", "attention_mask"],
    label_cols=["labels", "next_sentence_label"],
    batch_size=TRAIN_BATCH_SIZE,
    shuffle=True,
    collate_fn=collater,
)

## 08. 모델 정의

In [14]:
from transformers import BertConfig

config = BertConfig.from_pretrained(MODEL_CHECKPOINT)

In [15]:
from transformers import TFBertForPreTraining

model = TFBertForPreTraining(config)

In [16]:
# 옵티마이저 정의 및 모델 컴파일
optimizer = keras.optimizers.Adam(learning_rate=LEARNING_RATE)
model.compile(optimizer=optimizer)

No loss specified in compile() - the model's internal loss computation will be used as the loss. Don't panic - this is a common way to train TensorFlow models in Transformers! To disable this behaviour please pass a loss argument, or explicitly pass `loss=None` if you do not want your model to compute a loss.


In [17]:
# 모델 학습
model.fit(train, validation_data=validation, epochs=MAX_EPOCHS)

484/484 [==============================] - 9845s 20s/step - loss: 8.3759 - val_loss: 8.6529


In [19]:
# 식별자로 모델 불러오기

# 모델 푸시
# model.push_to_hub("pretrained-bert", organization="keras-io")
# tokenizer.push_to_hub("pretrained-bert", organization="keras-io")

# 모델 불러오기 1
# from transformers import TFBertForPreTraining
# model = TFBertForPreTraining.from_pretrained("your-username/my-awesome-model")

# 모델 불러오기 2
# from transformers import TFBertForSequenceClassification
# model = TFBertForSequenceClassification.from_pretrained("your-username/my-awesome-model")

# 이렇게 모델을 불러오면 사전 훈련 헤드가 삭제되고 모델은 변환기 레이어로 초기화되며 새로운 작업별 헤드가 임의의 가중치로 추가된다